In [8]:
!pip install google-generativeai python-dotenv SQLAlchemy

  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
Using cached google_generativeai-0.8.5-py3-none-any.whl (155 kB)


In [15]:
# --- 1. SETUP AND INSTALLATIONS ---

import google.generativeai as genai
import os
import json
from dotenv import load_dotenv
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker
# --- 2. CONFIGURATION AND KEY LOADING ---
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

JSON_TRANSCRIPT_PATH = r"C:\Users\apran\Videos\Cin\LIBRARY\Meeting Agent\transcript.json"
DATABASE_URL = "sqlite:///tasks.db"

# --- 3. DATABASE DEFINITION ---
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

class Task(Base):
    __tablename__ = "tasks"
    id = Column(Integer, primary_key=True, index=True)
    description = Column(String, index=True)
    assignee = Column(String)
    due_date_str = Column(String)
    status = Column(String, default="To Do")

Base.metadata.create_all(bind=engine)
print("✅ Database is ready.")

# --- 4. HELPER FUNCTIONS ---
def load_transcript_from_json(file_path):
    """Loads and formats the transcript from your JSON file."""
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        formatted_transcript = f"Meeting Title: {data['meeting_title']}\nParticipants: {', '.join(data['participants'])}\n\n"
        for entry in data['transcript']:
            formatted_transcript += f"{entry['speaker']} ({entry['timestamp']}): {entry['dialogue']}\n"
        return formatted_transcript
    except Exception as e:
        print(f"❌ Error loading transcript: {e}")
        return None

def process_transcript_with_gemini(transcript):
    """Generates MoM and extracts tasks using the Gemini API."""
    if not GEMINI_API_KEY:
        print("❌ Gemini API Key not found. Please check your .env file.")
        return None
    
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-2.5-flash')
    
    prompt = f"""
    Analyze the following meeting transcript. Your tasks are to:
    1.  Generate a concise "Minutes of Meeting" summary, capturing the key findings and decisions.
    2.  Extract all action items into a structured list.

    Respond with a single, valid JSON object with two keys: "minutes" (a string summary) and "tasks" (a list of objects).
    Each task object must have three keys: "task_description", "assignee", and "due_date". Use the information from the transcript to determine these values.

    Transcript:\n---\n{transcript}\n---
    """
    
    try:
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                response_mime_type="application/json"
            )
        )
        return json.loads(response.text)
    except Exception as e:
        print(f"❌ An error occurred with the Gemini API: {e}")
        return None

# --- 5. MAIN EXECUTION ---
print("Loading transcript...")
meeting_transcript = load_transcript_from_json(JSON_TRANSCRIPT_PATH)

tasks_to_save = []
if meeting_transcript:
    print("Processing with Gemini API...")
    ai_results = process_transcript_with_gemini(meeting_transcript)
    
    if ai_results:
        print("\n--- 📝 Generated Minutes of Meeting ---")
        print(ai_results.get("minutes", "No summary generated."))
        print("\n--- 📌 Extracted Action Items ---")
        tasks_to_save = ai_results.get("tasks", [])
        if tasks_to_save:
            for task in tasks_to_save:
                print(f"- {task}")
        else:
            print("No action items were extracted.")

# --- 6. UPDATE AND DISPLAY DATABASE ---
db = SessionLocal()
try:
    # Clear existing tasks to avoid duplicates on re-runs
    num_deleted = db.query(Task).delete()
    if num_deleted > 0:
        print(f"\n🗑️ Cleared {num_deleted} old tasks from the database.")
    
    # Add new tasks
    for task_item in tasks_to_save:
        new_task = Task(
            description=task_item.get("task_description"),
            assignee=str(task_item.get("assignee", "N/A")), # Ensure assignee is a string
            due_date_str=task_item.get("due_date"),
        )
        db.add(new_task)
    db.commit()
    if tasks_to_save:
        print(f"✅ Successfully saved {len(tasks_to_save)} new tasks to the database.")

    # Read all tasks back from the database to confirm
    print("\n--- 📖 Current Tasks in Database ---")
    all_tasks_in_db = db.query(Task).all()
    if not all_tasks_in_db:
        print("No tasks are currently in the database.")
    else:
        for i, task in enumerate(all_tasks_in_db, 1):
            print(f"  {i}. Task:      {task.description}")
            print(f"     Assignee:  {task.assignee}")
            print(f"     Due Date:  {task.due_date_str}")
            print(f"     Status:    {task.status}\n")
finally:
    db.close()

✅ Database is ready.
Loading transcript...
Processing with Gemini API...

--- 📝 Generated Minutes of Meeting ---
The Quarterly Oceanographic Research Sync focused on updates for the Bay of Bengal biogeochemical cycling project. Key findings included the discovery of a significant, previously unmapped chlorophyll-a plume at the 50-meter depth contour. This finding was corroborated by a 15% higher particulate organic carbon flux at that depth and a distinct microbial community with an increased abundance of nitrifying bacteria. The plume's location correlates with a persistent cyclonic eddy, suggesting upwelling of deep nutrients as a physical driver. The team decided to plan a new research cruise for early May to capture pre-monsoon conditions and conduct further investigation. An interdisciplinary plan was outlined involving new ARGO float deployments, biogeochemical model re-runs, historical satellite altimetry analysis of eddies, and metagenomic sequencing of water samples. The next 

In [16]:
def analyze_meeting_sentiment(transcript):
    """
    Analyzes the transcript to determine the overall sentiment of the meeting.
    """
    if not GEMINI_API_KEY:
        print("❌ Gemini API Key not found.")
        return None
    
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('models/gemini-2.5-flash')
    
    prompt = f"""
    Analyze the sentiment of the following meeting transcript.
    Classify the overall sentiment as "Positive", "Negative", or "Neutral".
    Provide a brief, one-sentence justification for your classification.

    Respond with a single JSON object with two keys: "sentiment" and "justification".

    Transcript:\n---\n{transcript}\n---
    """
    
    try:
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                response_mime_type="application/json"
            )
        )
        return json.loads(response.text)
    except Exception as e:
        print(f"❌ An error occurred with the Gemini API: {e}")
        return None

# --- Execution ---
print("🤔 Analyzing Meeting Sentiment...")
if 'meeting_transcript' in locals() and meeting_transcript:
    sentiment_results = analyze_meeting_sentiment(meeting_transcript)
    if sentiment_results:
        sentiment = sentiment_results.get('sentiment', 'Unknown')
        justification = sentiment_results.get('justification', 'No justification provided.')
        
        emoji = "😐"
        if sentiment == "Positive":
            emoji = "😊"
        elif sentiment == "Negative":
            emoji = "😠"
            
        print(f"\n--- 📊 Meeting Sentiment ---")
        print(f"Overall Sentiment: {sentiment} {emoji}")
        print(f"Justification: {justification}")
else:
    print("Please run the initial cell to load the transcript first.")

🤔 Analyzing Meeting Sentiment...

--- 📊 Meeting Sentiment ---
Overall Sentiment: Positive 😊
Justification: The transcript highlights exciting new discoveries, strong interdisciplinary collaboration, and a clear, well-received plan for future research, punctuated by positive affirmations from the project lead.


In [17]:
def extract_key_entities(transcript):
    """
    Extracts key entities like projects, people, and concepts from the transcript.
    """
    if not GEMINI_API_KEY:
        print("❌ Gemini API Key not found.")
        return None
    
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('models/gemini-2.5-flash')
    
    prompt = f"""
    Act as a data analyst. Your task is to extract key entities from the following meeting transcript.
    Organize these entities into four categories: "Projects", "Key_Concepts", "Technologies", and "People".
    
    Respond with a single JSON object where each key is a category, and the value is a list of the extracted entities (strings).

    Transcript:\n---\n{transcript}\n---
    """
    
    try:
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                response_mime_type="application/json"
            )
        )
        return json.loads(response.text)
    except Exception as e:
        print(f"❌ An error occurred with the Gemini API: {e}")
        return None

# --- Execution ---
print("🏷️ Extracting Key Entities and Keywords...")
if 'meeting_transcript' in locals() and meeting_transcript:
    entity_results = extract_key_entities(meeting_transcript)
    if entity_results:
        print(f"\n--- 🔑 Key Entities & Concepts ---")
        for category, items in entity_results.items():
            print(f"\n## {category.replace('_', ' ')}")
            if items:
                print(", ".join(items))
            else:
                print("None identified.")
else:
    print("Please run the initial cell to load the transcript first.")

🏷️ Extracting Key Entities and Keywords...

--- 🔑 Key Entities & Concepts ---

## Projects
Bay of Bengal biogeochemical cycling project

## Key Concepts
chlorophyll-a plume, sediment trap analysis, particulate organic carbon flux, physical drivers, cyclonic eddy, upwelling, nutrients, microbial community structure, nitrifying bacteria, nitrogen cycling, upwelling nutrient hypothesis, hydrographic team, temporal data, temperature, salinity, chlorophyll structure, data assimilation, life cycle of eddies, seasonal or random event, water samples, genes, physical data, chemical data, biological data, research cruise, pre-monsoon conditions, research vessel team, float deployments, CTD cast, model results, historical eddy analysis, sampling track, sampling protocols, preservation kits, microbial work, genetic work, cruise plan, data processing

## Technologies
ARGO floats, biogeochemical models, satellite altimetry, metagenomic sequencing, CTD

## People
Priya, Raghav, Anjali, Arjun, Meena
